# Traffic sign classifier

## Machine Learning Project: Build a Traffic Sign Classifier

---
**Disclaimer**

This project aims to be a study and a reworking of [this](https://github.com/lfiaschi/udacity-traffic-sign-classifier) notebook.

In [32]:
import cv2 # resize the images
import numpy as np
import pandas as pd
import os # to work with directories
from random import shuffle # shuffle data


DATASET_DIR = '/datasets/GTSRB/Images/'
ANNOTATION_FILE = './signnames.csv'



# IMAGE_SIZE = 50
# LR = 1e-3
# MODEL_NAME = 'trafficsigns--{}--{}.model'.format(LR, "2conv-basic")

Load the csv file with the annotations

In [33]:
signnames = pd.read_csv(ANNOTATION_FILE)
signnames.set_index('ClassId',inplace=True) 
print(signnames[:5])

def get_name_from_label(label):
    """Return the textual name corresponding to the numeric class id
    
    this functions look for a correspondence between the numeric class id 
    and the textual name of the class.
    
    :param label: the numeric class id
    :type label: int
    :returns: the textual name of the class
    
    :Example:

    >>> get_name_from_label(0)
    Speed limit (20km/h)   
    """    
    
    return signnames.loc[label].SignName

                     SignName
ClassId                      
0        Speed limit (20km/h)
1        Speed limit (30km/h)
2        Speed limit (50km/h)
3        Speed limit (60km/h)
4        Speed limit (70km/h)


The images are divided into folders based on their category. The *load_dataset* function create a list of all the images labeled with the name of their folder.

In [47]:

def load_dataset(path):
    """Load a dataset of images given a path
    
    this function look for images on the subfolders of the given path and label 
    them with the name of the folder where the image is stored
    
    :param path: the path where the images divided into folders are stored
    :returns: a numpy array    
    """
    dataset = []
    for subdir, dirs, files in os.walk(path): # all file on the dataset folder
        for img in files: # one image by one
            
            filename, file_extension = os.path.splitext(img) #extension control
            if file_extension == '.ppm':
                label = os.path.basename(subdir) # obtain the img label (name of the folder)
                imgPath = os.path.join(path, label, img) # the path of the img
                #img = cv2.Load_im(path, cv2.IMREAD_GRAYSCALE), (IMAGE_SIZE, IMAGE_SIZE))
                
                #load image with cv2 library
                img = cv2.imread(imgPath,0)
                
                #trasform into np array
                imgNp = np.asarray(img)
                
                label = int(label) # remove the zeros ahead the name of the folder
                #training_data.append([np.array(img), np.array(label)])
                dataset.append([imgNp, label])
             
           
    shuffle(dataset)
    # np.save('dataset.npy', dataset)
    return dataset

In [48]:
#stampa del dataset sotto forma di numpy array. (giusto?)
load_dataset('/datasets/GTSRB/Images/')

[[array([[252, 187, 209, ...,  23,  24,  28],
         [249, 183, 208, ...,  22,  24,  28],
         [252, 180, 210, ...,  23,  24,  27],
         ..., 
         [ 22,  21,  21, ..., 126, 255, 207],
         [ 32,  32,  32, ..., 123, 255, 209],
         [ 29,  27,  27, ..., 118, 244, 209]], dtype=uint8), 15],
 [array([[10, 11, 10, ..., 11, 11, 12],
         [10,  9, 11, ..., 10, 11, 11],
         [10,  9, 10, ...,  9, 10,  9],
         ..., 
         [10,  8,  9, ..., 10, 13, 14],
         [10,  8,  9, ...,  9, 11, 13],
         [10,  8,  9, ...,  9, 11, 10]], dtype=uint8), 33],
 [array([[255, 255, 255, ...,  52,  49,  48],
         [255, 255, 255, ...,  50,  49,  47],
         [255, 255, 255, ...,  46,  46,  47],
         ..., 
         [ 65,  55,  51, ...,  74,  77,  78],
         [ 56,  53,  51, ...,  79,  80,  80],
         [ 52,  50,  48, ...,  79,  82,  83]], dtype=uint8), 11],
 [array([[177, 160, 167, ..., 108, 117, 125],
         [163, 146, 122, ...,  95, 108, 120],
         [1

In [6]:
dataset = load_dataset(DATASET_DIR)
print("dataset cardinality : {}".format(len(dataset)))

# print the first 5 elements
for data in dataset[0:5]:
    print(data[0], "\t", data[1], "\t", get_name_from_label(data[1]))

dataset cardinality : 39252
/datasets/GTSRB/Images/00015/00016_00010.ppm 	 15 	 No vehicles
/datasets/GTSRB/Images/00035/00035_00025.ppm 	 35 	 Ahead only
/datasets/GTSRB/Images/00022/00005_00025.ppm 	 22 	 Bumpy road
/datasets/GTSRB/Images/00004/00047_00003.ppm 	 4 	 Speed limit (70km/h)
/datasets/GTSRB/Images/00015/00003_00011.ppm 	 15 	 No vehicles


TODO cercare di ottenere X_train, y_train, X_valid, y_valid per allienarci con il progetto originale

## Select data to create the trainingset and the testset

In [16]:
# training_set = dataset[:-5000] # take all the images except the last 5000
# test_set = dataset[-5000:] # take the last 5000 images
# print("training set cardinality : {}".format(len(training_set)))
# print("testset cardinality : {}".format(len(test_set)))

training set cardinality : 34252
testset cardinality : 5000
